In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import user_agent
from user_agent import generate_user_agent, generate_navigator

# 웹드라이버 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument('lang=ko_KR') # 사용언어 한국어
chrome_options.add_argument('disable-gpu') # 하드웨어 가속 안함
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
#chrome_options.add_argument('headless') # 창 숨기기

In [4]:
#### 처음 제목, 링크, 날짜 리스트에 담기
#셀레니움 스크롤 끝까지 내려도 계속 내리는 페이지라면
def scroll_first():
    prev_height = driver.execute_script("return document. body.scrollHeight")

    while True:
        #첫번째로 스크롤 내리기
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        #시간대기
        time.sleep(2)
        #현재높이 저장
        current_height = driver.execute_script("return document. body.scrollHeight")
        #현재높이와 끝의 높이가 같으면 탈출
        if(current_height == prev_height):
            break
        else:
            prev_height = driver.execute_script("return document.body.scrollHeight")

    # 검색 결과 블로그글 url과 제목 가져오기
    class_articles = ".api_txt_lines.total_tit"
    url_link = driver.find_elements(By.CSS_SELECTOR, class_articles)
    # 글 작성 일자 가져오기
    class_datetime = ".sub_time.sub_txt"
    date_time = driver.find_elements(By.CSS_SELECTOR, class_datetime)
    
    global url_list
    global title_list
    global date_list

    url_list = []
    title_list = []
    date_list = []

    for article in url_link:
        url = article.get_attribute('href')
        url_list.append(url)
    for article in url_link:
        title = article.text
        title_list.append(title)
    for date in date_time:
        datetime = date.text
        date_list.append(datetime)

#셀레니움 스크롤 끝까지 내려도 계속 내리는 페이지라면
def scroll_next():
    prev_height = driver.execute_script("return document. body.scrollHeight")

    while True:
        #첫번째로 스크롤 내리기
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        #시간대기
        time.sleep(2)
        #현재높이 저장
        current_height = driver.execute_script("return document. body.scrollHeight")
        #현재높이와 끝의 높이가 끝이면 탈출
        if(current_height == prev_height):
            break
        else:
            prev_height = driver.execute_script("return document.body.scrollHeight")

    # 검색 결과 블로그글 url과 제목 가져오기
    class_articles = ".api_txt_lines.total_tit"
    url_link = driver.find_elements(By.CSS_SELECTOR, class_articles)
    # 글 작성 일자 가져오기
    class_datetime = ".sub_time.sub_txt"
    date_time = driver.find_elements(By.CSS_SELECTOR, class_datetime)

    for article in url_link:
        url = article.get_attribute('href')
        url_list.append(url)
    for article in url_link:
        title = article.text
        title_list.append(title)
    for date in date_time:
        datetime = date.text
        date_list.append(datetime)

# year=연도(연도에 지정된 숫자! 2018년은 16), month=월(1~12), days=해당 월의 일수
def basic_crawling(year, month, days):
    day_list = list(range(days,0,-1))
    
    driver.find_element(By.LINK_TEXT, "VIEW").click()
    time.sleep(2)
    driver.find_element(By.LINK_TEXT, "블로그").click()
    time.sleep(2)
    driver.find_element(By.LINK_TEXT, "옵션").click()
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.lineup > div > div > a:nth-child(2)").click()
    time.sleep(2)
    
    # 옵션 > 기간 >  직접입력 클릭
    driver.find_element(By.CSS_SELECTOR, "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a.txt.txt_option._calendar_select_trigger").click()
    time.sleep(2)

    # 기간 직접입력 시작과 끝의 앞부분 동일 CSS 코드 변수로 담기 (코드 단순화용)
    GS_head_css = "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > "
    # 기간 직접입력 년월일 앞 CSS 코드 변수로 담기 (코드 단순화용)
    GSYMD_head_css = "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > "
    
    for day in day_list:
        if day == days:
            print('{}일 작성글 검색 후 수집중 . . .'.format(day))
        #  ★☆★☆ 맨 처음 기간설정 할 때 ☆★☆★
            #### 기간 설정시작 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.set_calendar > span:nth-child(1) > a").click()
            time.sleep(1)
            # 연도는 year
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(1) > div > div > div > ul > li:nth-child({})".format(year-2)).click()
            time.sleep(1)
            # 월은 month
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(2) > div > div > div > ul > li:nth-child({})".format(month)).click()
            time.sleep(1)
            # 일자는 days. 마지막 일자 부터 검색해서 저장할 것이기 때문!
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(3) > div > div > div > ul > li:nth-child({})".format(days)).click()
            time.sleep(1)

            #### 기간 설정끝 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.set_calendar > span:nth-child(3) > a").click()
            time.sleep(1)
            # 연도는 year
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(1) > div > div > div > ul > li:nth-child({})".format(year-2)).click()
            time.sleep(1)
            # 월은 month
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(2) > div > div > div > ul > li:nth-child({})".format(month)).click()
            time.sleep(1)
            # 일자는 days. 마지막 일자 부터 검색해서 저장할 것이기 때문!
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(3) > div > div > div > ul > li:nth-child({})".format(days)).click()
            time.sleep(1)

            #### 기간 설정란 적용 버튼 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.btn_area > button").click()
            time.sleep(1)
            
            # 첫 번째 스크롤&전역변수 선언&제목/링크/날짜 저장
            scroll_first()
            time.sleep(2)

        
        else:
        # 두 번째부터는 스크롤 맨 위로 올려서 일자만 바꾸기!
            print('{}일 작성글 검색 후 수집중 . . .'.format(day))
            # 스크롤 맨 위로 올리기
            driver.execute_script('window.scrollTo(0,0)')
            time.sleep(1)
            # 옵션 > 기간 >  직접입력 클릭
            driver.find_element(By.CSS_SELECTOR, "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a.txt.txt_option._calendar_select_trigger").click()
            driver.implicitly_wait(10)
            time.sleep(1)
            # 기간 설정시작 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.set_calendar > span:nth-child(1) > a").click()
            driver.implicitly_wait(10)
            time.sleep(1)
            # # 일자는 day_list에서 for 문 돌면서 i번째 숫자 가져오기
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(3) > div > div > div > ul > li:nth-child({})".format(day)).click()
            driver.implicitly_wait(10)
            time.sleep(1)
            # 기간 설정끝 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.set_calendar > span:nth-child(3) > a").click()
            driver.implicitly_wait(10)
            time.sleep(1)
            # # 일자는 day_list에서 for 문 돌면서 i번째 숫자 가져오기
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(3) > div > div > div > ul > li:nth-child({})".format(day)).click()
            # 기간 설정란 적용 버튼 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.btn_area > button").click()
            driver.implicitly_wait(10)
            time.sleep(1)
            
            # 두 번째 스크롤&제목/링크/날짜 저장
            scroll_next()
            
    global crawl_df
    crawl_df = pd.DataFrame({'url': url_list, 'title':title_list, 'date':date_list})

In [ ]:
# 크롤링 할 기간 설정.(역순 크롤링)
Yr = 22 # 원하는 연도설정
M1 = 7 # 원하는 시작 월 설정
M2 = 7 # 원하는 끝 월 설정

for i in range(M1,M2-1,-1): # M1부터 M2까지 차례대로 반복
    navigator = generate_navigator()
    # random_user = generate_user_agent(os='win', device_type='desktop') # 랜덤한 헤더 생성
    # chrome_options.add_argument(random_user)
    driver = webdriver.Chrome(executable_path ='../chromedriver_win32/chromedriver.exe',options = chrome_options)
    driver.get("https://www.naver.com/")
    driver.implicitly_wait(10)
    time.sleep(2)
    query_text = '"나무" +"문화"'

    element = driver.find_element(By.ID, "query");

    element.send_keys(query_text)
    element.submit()
    driver.implicitly_wait(10)
    time.sleep(2)
    if i in [1,3,5,7,8,10,12]:
        basic_crawling(Yr,i,31)
    elif i in [4,6,9,11]:
        basic_crawling(Yr,i,30)
    elif i == 2 and Yr%4 == 0: # 윤년을 고려하여 2월 29일인 케이스 추가
         basic_crawling(Yr,i,29)
    elif i == 2 and Yr%4 != 0: # 윤년이 아닌 2월일 경우
        basic_crawling(Yr,i,28)
    crawl_df.drop_duplicates(inplace=True) #중복제거
    crawl_df = crawl_df.reset_index(drop=True) #인덱스 초기화 

    link_list = crawl_df['url'].to_list()
    find_naver = [i for i in range(len(link_list)) if 'blog.naver.com' in link_list[i]] # 네이버블로그만 남기기 위해 재검사.

    NB_DF = crawl_df.iloc[find_naver]
    print(len(NB_DF))
    #NB_DF.to_csv("../나무문화/%s/나무문화_링크_%s_%s월.csv"%("20"+str(Yr),"20"+str(Yr),i), index=False, encoding='UTF-8')
    driver.quit()
    print(i)

In [10]:
i

7

In [11]:
NB_DF.to_csv("../dataset/%s/링크_크롤링/나무문화_링크_%s_%s월.csv"%("20"+str(Yr),"20"+str(Yr),i), index=False, encoding='UTF-8')

In [18]:
Yr

20